In [10]:
import torch
import torchvision

In [15]:
path_train_dataset = "C:\\Users\\alexi\\PROJET_3A\\imagenette2-160\\train"
path_valid_dataset = "C:\\Users\\alexi\\PROJET_3A\\imagenette2-160\\val"
n_classes = 10
batch_size = 8
n_epochs = 1
path_best_model = "imagenette2-160_model.pth"

In [11]:
train_dataset =  torchvision.datasets.ImageFolder(
    path_train_dataset,
    torchvision.transforms.Compose([
        torchvision.transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

valid_dataset = torchvision.datasets.ImageFolder(
    path_valid_dataset,
    torchvision.transforms.Compose([
        torchvision.transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        torchvision.transforms.Resize((224, 224)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

valid_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
)

In [16]:
model = torchvision.models.alexnet(pretrained=True)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, n_classes)

c:\Users\alexi\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\alexi\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [23]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

best_accuracy = 0.0

for epoch in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader) :
        optimizer.zero_grad()
        outputs = model(images)
        loss = torch.nn.functional.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
        print('epoch %d batch %d/%d' % (epoch, (i + 1), len(train_loader)))
    test_error_count = 0.0
    for images, labels in valid_loader:
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
        
    test_accuracy = 1.0 - float(test_error_count) / float(len(valid_dataset))
    print('epoch %d : test_accuracy %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), path_best_model)
        best_accuracy = test_accuracy

epoch 0 batch 1/1184
epoch 0 batch 2/1184
epoch 0 batch 3/1184
epoch 0 batch 4/1184
epoch 0 batch 5/1184
epoch 0 batch 6/1184


KeyboardInterrupt: 